In [1]:
from sympy import *
import math

In [2]:
3 + math.sqrt(3)

4.732050807568877

In [3]:
expr = 3 + sqrt(3)
expr

sqrt(3) + 3

In [4]:
init_printing(use_latex="mathjax")

In [5]:
expr

√3 + 3

## Symbols

In [32]:
x, y = symbols("x y")

In [33]:
dx, dy= symbols("dx dy")

## Funções de Forma

In [36]:
N1 = (1-x/dx)*(1-y/dy)
N2 = (1-x/dx)*(0+y/dy)
N3 = (0+x/dx)*(0+y/dy)
N4 = (0+x/dx)*(1-y/dy)

In [37]:
N1

⎛    x ⎞ ⎛    y ⎞
⎜1 - ──⎟⋅⎜1 - ──⎟
⎝    dx⎠ ⎝    dy⎠

## Matriz de Elasticidade

In [39]:
E, v = symbols("E v")
Dinv = (1/E)*Matrix([[1,-v,0], [-v,1,0], [0, 0,2*(1+v)]])
D = simplify(Dinv**-1)

In [40]:
D

⎡ -E     -E⋅v             ⎤
⎢──────  ──────      0    ⎥
⎢ 2       2               ⎥
⎢v  - 1  v  - 1           ⎥
⎢                         ⎥
⎢-E⋅v     -E              ⎥
⎢──────  ──────      0    ⎥
⎢ 2       2               ⎥
⎢v  - 1  v  - 1           ⎥
⎢                         ⎥
⎢                    E    ⎥
⎢  0       0     ─────────⎥
⎣                2⋅(v + 1)⎦

In [41]:
A = Matrix([[1,2],[3,4],[5,6]])
A.shape


(3, 2)

In [42]:
def applyS(N):

    ddx = diff(N, x)
    ddy = diff(N, y)
    ddz = diff(N, z)
    
    return Matrix([[ddx, 0], [0, ddy],  [ddy, ddx]])

In [43]:
Stiff = applyS(N1).T*D*applyS(N2)
Aux = zeros(2,2)
for i in range(Stiff.shape[0]*Stiff.shape[1]):
    Aux[i] = Integral(Stiff[i],(x, 0, dx))
    Aux[i] = Integral(Aux[i],(y, 0, dy))
Stiff = Aux
Stiff

⎡   dy dx                                                                     
⎢   ⌠  ⌠                                                dy dx                 
⎢   ⎮  ⎮  ⎛             2                   ⎞           ⌠  ⌠                  
⎢   ⎮  ⎮  ⎜     ⎛    x ⎞           ⎛    y ⎞ ⎟           ⎮  ⎮  ⎛    ⎛    x ⎞ ⎛ 
⎢   ⎮  ⎮  ⎜   E⋅⎜1 - ──⎟       E⋅y⋅⎜1 - ──⎟ ⎟           ⎮  ⎮  ⎜E⋅v⋅⎜1 - ──⎟⋅⎜1
⎢   ⎮  ⎮  ⎜     ⎝    dx⎠           ⎝    dy⎠ ⎟           ⎮  ⎮  ⎜    ⎝    dx⎠ ⎝ 
⎢   ⎮  ⎮  ⎜- ───────────── - ───────────────⎟ dx dy     ⎮  ⎮  ⎜───────────────
⎢   ⎮  ⎮  ⎜      2             2    ⎛ 2    ⎞⎟           ⎮  ⎮  ⎜          ⎛ 2  
⎢   ⎮  ⎮  ⎝  2⋅dy ⋅(v + 1)   dx ⋅dy⋅⎝v  - 1⎠⎠           ⎮  ⎮  ⎝    dx⋅dy⋅⎝v  -
⎢   ⌡  ⌡                                                ⌡  ⌡                  
⎢   0  0                                                0  0                  
⎢                                                                             
⎢                                                   

In [44]:
E_value = 1
v_value = 0.25
dx_value = 10000
dy_value = 10000
dz_value = 10000

In [15]:
def CalcConnectionAnalitic(N1, N2):
    Stiff = applyS(N1).T*D*applyS(N2)
    
    Aux = zeros(Stiff.shape[0], Stiff.shape[1])
    
    for i in range(Stiff.shape[0]*Stiff.shape[1]):
        Aux[i] = Integral(Stiff[i],(x, 0, dx))
        Aux[i] = Integral(Aux[i],(y, 0, dy))
        Aux[i] = Integral(Aux[i],(z, 0, dz))
        Aux[i] = simplify(Aux[i])

    Stiff = Aux
   
    return Stiff     

In [16]:
def CalcConnection(N1, N2):
    Stiff = applyS(N1).T*D*applyS(N2)
    
    Aux = zeros(Stiff.shape[0], Stiff.shape[1])
    
    for i in range(Stiff.shape[0]*Stiff.shape[1]):
        Aux[i] = Integral(Stiff[i],(x, 0, dx))
        Aux[i] = Integral(Aux[i],(y, 0, dy))
        Aux[i] = Integral(Aux[i],(z, 0, dz))
    
    Stiff = Aux
   
    for i in range(Stiff.shape[0]*Stiff.shape[1]):
        Aux[i]=Stiff[i].doit().subs(E, E_value).subs(v, v_value).subs(dx, dx_value).subs(dy, dy_value).subs(dz, dz_value)

    Stiff = Aux
    return Stiff  

In [17]:
def CalcConnection(N1, N2):
    Stiff = CalcConnectionAnalitic(N1, N2)
    
    Aux = zeros(Stiff.shape[0], Stiff.shape[1])
      
    for i in range(Stiff.shape[0]*Stiff.shape[1]):
        Aux[i]=Stiff[i].doit().subs(E, E_value).subs(v, v_value).subs(dx, dx_value).subs(dy, dy_value).subs(dz, dz_value)

    Stiff = Aux
    return Stiff  

In [18]:
proprio = CalcConnection(N1, N1)
for func in [N2, N3, N4, N5, N6, N7, N8]:
    proprio += CalcConnection(func, func)

In [19]:
acima = CalcConnection(N1, N2) + CalcConnection(N4, N3) + CalcConnection(N5, N6) + CalcConnection(N8, N7)

In [20]:
sul = CalcConnection(N5, N1) + CalcConnection(N6, N2) + CalcConnection(N7, N3) + CalcConnection(N8, N4)

In [21]:
proprio

⎡41700960219.4788         0                 0        ⎤
⎢                                                    ⎥
⎢       0          41700960219.4788         0        ⎥
⎢                                                    ⎥
⎣       0                 0          41700960219.4787⎦

In [22]:
acima

⎡5486968449.93141         0                  0        ⎤
⎢                                                     ⎥
⎢       0          5486968449.93141          0        ⎥
⎢                                                     ⎥
⎣       0                 0          -10973936899.8628⎦

In [23]:
sul

⎡5486968449.93142          0                 0        ⎤
⎢                                                     ⎥
⎢       0          -10973936899.8628         0        ⎥
⎢                                                     ⎥
⎣       0                  0          5486968449.93141⎦

In [24]:
acimaAnalitica = CalcConnectionAnalitic(N1, N2) + CalcConnectionAnalitic(N4, N3) + CalcConnectionAnalitic(N5, N6) + CalcConnectionAnalitic(N8, N7)

In [25]:
acimaAnalitica = simplify(acimaAnalitica.doit())

In [26]:
acimaAnalitica

⎡  ⎛      2   2 ⎛   2        ⎞     2         ⎛    2       2       2         2⎞
⎢E⋅⎝- 2⋅dx ⋅dy ⋅⎝2⋅v  + v - 1⎠ + dz ⋅(v + 1)⋅⎝2⋅dx ⋅v - dx  + 2⋅dy ⋅v - 2⋅dy ⎠
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                         ⎛   2        ⎞                      
⎢                      9⋅dx⋅dy⋅dz⋅(v + 1)⋅⎝2⋅v  + v - 1⎠                      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                      0                                      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [27]:
sulAnalitica = CalcConnectionAnalitic(N5, N1) + CalcConnectionAnalitic(N6, N2) + CalcConnectionAnalitic(N7, N3) + CalcConnectionAnalitic(N8, N4)
sulAnalitica = simplify(sulAnalitica.doit())

In [28]:
sulAnalitica

⎡  ⎛    2   2       2   2       2   2         2   2       2   2         2   2⎞
⎢E⋅⎝2⋅dx ⋅dy ⋅v - dx ⋅dy  - 4⋅dx ⋅dz ⋅v + 2⋅dx ⋅dz  + 2⋅dy ⋅dz ⋅v - 2⋅dy ⋅dz ⎠
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                     ⎛   2        ⎞                          
⎢                          9⋅dx⋅dy⋅dz⋅⎝2⋅v  + v - 1⎠                          
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                      0                                      
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [29]:
a = acimaAnalitica[0]/sulAnalitica[0]
a = a.subs(dx, 1).subs(dy, 1)
a


  2                        2          
dz ⋅(v + 1)⋅(4⋅v - 3) - 4⋅v  - 2⋅v + 2
──────────────────────────────────────
            ⎛      2            ⎞     
    (v + 1)⋅⎝- 2⋅dz ⋅v + 2⋅v - 1⎠     

In [30]:
limit(a, dz, 0)

-2